In [1]:
#Load related libraries to do analysis
import pandas as pd #To work with dataset
import numpy as np #Math library

In [2]:
df_acquisition = pd.read_table('output/fm2000Q1_acquisition_sample.txt', sep = '|')
df_performance = pd.read_table('output/fm2000Q1_performance_sample.txt', sep = '|')

In [5]:
# Create MOB of all even if account is gone from newer business date
# The use of this methodology depend whether the data gone or not
# The MOB based on tenor

# Choose select columns only to create
df_performance2 = df_performance[['LOAN_ID','ORIG_TRM']]

df_create = pd.concat([pd.DataFrame({'LOAN_ID':row.LOAN_ID,
                                     'MOB':range(0,(row.ORIG_TRM+1)),
                                     'ORIG_TRM':row.ORIG_TRM
                                    },
                                    columns = ['LOAN_ID','MOB','ORIG_TRM'])
                                    for i, row in df_performance2.iterrows()], ignore_index=True)
# a must to remove duplicate due to how above code works
df_create.drop_duplicates(subset = ['LOAN_ID','MOB'], keep = 'first', inplace=True)

# recommended to have, if you know that you only want to track up to MOB 90
#df_create = df_create[df_create.MOB <= 60]


In [7]:
df_performance2 = df_performance[['LOAN_ID','Delq.Status','Loan.Age']]
print(df_performance2.info())
print(df_performance2['Delq.Status'].unique())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56932 entries, 0 to 56931
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   LOAN_ID      56932 non-null  int64 
 1   Delq.Status  56919 non-null  object
 2   Loan.Age     56932 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 1.3+ MB
None
['0' 'X' '1' '2' '3' '4' '5' '6' '7' '8' '9' '10' '11' '12' '13' '14' nan
 '15' '16' '17' '18' '19' '20' '21' '22' '23' '24' '25' '26' '27' '28'
 '29' '30' '31' '32' '33' '34' '35' '36' '37' '38' '39' '40' '41']


In [13]:
# Create list of loan ID that ever DPD30+ (kolek 1) or DPD90+ (kolek 2) and the MOB when it happened
df_performance2 = df_performance[['LOAN_ID','Delq.Status','Loan.Age']]
df_performance2 = df_performance2[df_performance2['Delq.Status'] != 'X']
df_performance2['Delq.Status'] = df_performance2['Delq.Status'].astype(float)

# Ever DPD30+
df_ever30 = df_performance2[df_performance2['Delq.Status'] >= 1]
df_ever30_2  = df_ever30.groupby(['LOAN_ID'], as_index = False).agg(
    {
        'Loan.Age':'min'
    }
)
df_ever30_2['EVER30'] = 1
df_ever30_2.rename(columns={'Loan.Age':'DPD30_When'}, inplace = True)

# Ever DPD30+
df_ever90 = df_performance2[df_performance2['Delq.Status'] >= 2]
df_ever90_2  = df_ever90.groupby(['LOAN_ID'], as_index = False).agg(
    {
        'Loan.Age':'min'
    }
)
df_ever90_2['EVER90'] = 1
df_ever90_2.rename(columns={'Loan.Age':'DPD90_When'}, inplace = True)

In [ ]:
# Create list of latest date
df_performance2 = df_performance[['LOAN_ID','LAST_UPB','Loan.Age','Monthly.Rpt.Prd']]
